# Analyze Maricopa County, Arizona  Biodiversity Intactness Index (BII) changes from 2017-2020

Author: Hazel Vaquero

Repository Link: https://github.com/hazelvaq/Maricopa-Arizona-Changes-in-Biodiversity-Intactness-Index-.git 

## About 

In this notebook we analyse the changes observed in Biodiversity Intactness Index (BII) in Maricopa County, Phoenix, Arizona from 2017 to 2020. In 2021, Maricopa County was reported as the fastest growing county in the United States since 2001. It is the 4th largest county in terms of population. Urban development creates pressure on biodiversity and natural areas. This notebook seeks to analyze the impact of the rapid urban expansion in Maricopa County. 

## About the Datasets

In this notebook we use two different datasets:

**1. BII Data** 

The first dataset is from the [Planetary Computer Impact Observatory](https://planetarycomputer.microsoft.com/dataset/io-biodiversity). This dataset estimates terrestrial Biodiversity Intactness as a 100-meter gridded maps for the years 2017-2020. Biodiversity intactness is an estimated combination of two metrics:
- Abundance
- Compositional Similarity: how similar the composition of species is to an intact baseline

The collection we will be working with is `io-biodiversity`. 

**2. Phoenix Subdivison** 

The second dataste is from the [United Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/). This data has been downloaded as a shapefile and has 19 columns. It contains the county names, land area and water area to name a few.    

## Final Output

The final visualization

### Import Libraries 

In [6]:
# Import Libraries
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

# Used to access STAC catalogs
from pystac_client import Client
# Used to sign items from the MPC STAC catalog
import planetary_computer

# other libraries for nice outputs
from IPython.display import Image

ModuleNotFoundError: No module named 'pystac_client'

### Import Data

In [2]:
# Data Import: BII data

## Acess Planatery Computer Catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## Catalog Search
# Set temporal range
time_range = "2017/2020"
# Bonding box of Phoenix subdivision
bbox = {
   "type": "Polygon",
    "coordinates": [
        [
            [-112.826843, 32.974108],
            [-111.184387, 32.974108],
            [-111.184387, 33.863574],
            [-112.826843, 33.863574],
            [-112.826843, 32.974108]  # Make sure the first and last coordinates are the same to close the polygon
        ]
    ]
}
# catalog search
search = catalog.search(
    collections=['io-biodiversity'],
    intersects=bbox,
    datetime = time_range
)

# Get items found in catalog search
item = search.item_collection()

# get first item in the catalog search
item = items[0]

In [13]:
# Data Import: Phoenix subdivision
phoenix_counties = gpd.read_file('data/tl_2022_04_cousub.shp')

## Prepare Data

In [24]:
phoenix_counties

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,04,005,91198,01934931,0400591198,Flagstaff,Flagstaff CCD,22,Z5,G4040,None,None,None,S,12231052883,44653332,+35.1066114,-111.3662497,"POLYGON ((-112.13370 35.85596, -112.13368 35.8..."
1,04,005,91838,01934953,0400591838,Kaibab Plateau,Kaibab Plateau CCD,22,Z5,G4040,None,None,None,S,7228864534,29327221,+36.5991097,-112.1368033,"POLYGON ((-112.66039 36.53941, -112.66033 36.5..."
2,04,005,91683,01934950,0400591683,Hualapai,Hualapai CCD,22,Z5,G4040,None,None,None,S,2342313339,3772690,+35.9271665,-113.1170408,"POLYGON ((-113.35416 36.04097, -113.35416 36.0..."
3,04,023,92295,01934961,0402392295,Nogales,Nogales CCD,22,Z5,G4040,None,None,None,S,1762322938,2382710,+31.4956020,-111.0171332,"POLYGON ((-111.36692 31.52136, -111.36316 31.5..."
4,04,023,92550,01934966,0402392550,Patagonia,Patagonia CCD,22,Z5,G4040,None,None,None,S,1439530300,685527,+31.5664619,-110.6410279,"POLYGON ((-110.96273 31.68695, -110.96251 31.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,04,012,92480,01934965,0401292480,Parker,Parker CCD,22,Z5,G4040,None,None,None,S,11646086560,36514347,+33.7276104,-114.0388060,"POLYGON ((-114.73122 33.30404, -114.72990 33.3..."
76,04,003,90255,01934922,0400390255,Benson,Benson CCD,22,Z5,G4040,None,None,None,S,2386116645,1180899,+32.0883089,-110.2481520,"POLYGON ((-110.45231 31.76021, -110.45231 31.7..."
77,04,003,90969,01934936,0400390969,Douglas,Douglas CCD,22,Z5,G4040,None,None,None,S,641280940,178341,+31.4446761,-109.4183013,"POLYGON ((-109.79275 31.33442, -109.79266 31.3..."
78,04,003,93182,01934980,0400393182,Sierra Vista,Sierra Vista CCD,22,Z5,G4040,None,None,None,S,1814147964,1074824,+31.7198887,-110.1649608,"POLYGON ((-110.46078 31.56927, -110.46078 31.5..."
